# Выгрузим из каталога grlist_s группы с $ Ntot > 30$

In [1]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1
grplist = Vizier.query_constraints(catalog="J/A+A/596/A14/grlist_s", Ntot=">30")[0]
grplist

iGrID,RAJ2000,DEJ2000,z,logLtot,logLobs,logMtot,logMstar,NMstar,logMdyn,sigma,Rad,angRad,DL,Ntot,Dist
,deg,deg,,[Lsun],[Lsun],[Msun],[Msun],,[Msun],km / s,kpc,deg,Mpc,,
int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,bytes1
51,258.84573364,57.41119385,0.0292310,11.81573,11.79229,14.37889,12.30336,67,11.9609,562.4061,414.3665,0.204412,123.0337,67,*
142,194.65371704,-1.79300356,0.0842295,12.27122,12.12088,14.73268,12.60806,51,12.2978,517.7142,1062.0715,0.192726,371.1736,51,*
157,195.28141785,-3.44793510,0.0860228,12.09200,11.93540,14.53491,12.49946,34,12.0636,480.1897,719.9396,0.128148,379.6519,34,*
184,10.75667191,15.20660210,0.0779504,12.18950,12.05950,14.60890,12.57019,44,12.1100,524.9848,670.2989,0.130605,341.6862,44,*
211,195.72035217,-2.52054167,0.0840534,12.47245,12.32273,14.99262,12.81219,91,12.4724,647.4734,1015.1303,0.184561,370.3425,91,*
362,27.29955101,14.05051517,0.0693623,11.96650,11.86043,14.35009,12.29983,34,11.6868,399.8179,436.1201,0.094664,301.8517,35,*
417,176.40602112,64.51252747,0.0636209,11.97746,11.88558,14.28361,12.20315,31,11.6532,326.9466,603.6110,0.141995,275.5365,32,*
632,170.52259827,0.78428489,0.1038390,12.36823,12.14250,14.86289,12.61491,43,12.5639,520.8827,1936.1392,0.290502,465.2843,43,*


# Выгрузим фотометрические данные, как показано в туториале

In [7]:
import time
from astroquery.sdss import SDSS

# Some clusters are huge and long time is required to proccessed query,
# so let's increase default timeout from 1 minute to 5 min
SDSS.TIMEOUT = 600

factor = 1.5

for idx, grp in enumerate(grplist):
    query = (
"""
SELECT
    g.objid, g.ra, g.dec, g. dered_g, g. dered_r, err_g, err_r, petroRad_r, petroRadErr_r
FROM
    Galaxy AS g,
    dbo.fGetNearbyObjEq({},{},{}) AS n 
WHERE
    g.objID = n.objID
    AND ((g.flags_r & 0x10000000) != 0)
        -- detected in BINNED1
    AND ((g.flags_r & 0x8100000c00a0) = 0)
        -- not NOPROFILE, PEAKCENTER, NOTCHECKED, PSF_FLUX_INTERP, SATURATED,
        -- or BAD_COUNTS_ERROR.
        -- if you want to accept objects with interpolation problems for PSF mags,
        -- change this to: AND ((flags_r & 0x800a0) = 0)
    AND (((g.flags_r & 0x400000000000) = 0) or (g.psfmagerr_r <= 0.2))
        -- not DEBLEND_NOPEAK or small PSF error
        -- (substitute psfmagerr in other band as appropriate)
    AND (((g.flags_r & 0x100000000000) = 0) or (g.flags_r & 0x1000) = 0)
        -- not INTERP_CENTER or not COSMIC_RAY - omit this AND clause if you want to
        -- accept objects with interpolation problems for PSF mags.
""".format(grp['RAJ2000'], grp['DEJ2000'], grp['angRad']*60.0*factor)
    )
    
    msg = "[{:d}/{:d}] Working on iGrID = {:<8d} (Ntot = {:<3d} Rad = {:<0.1f} arcmin) at RA={}; DEC={}"
    print(msg.format(idx+1, len(grplist), grp['iGrID'], grp['Ntot'],
                     grp['RAJ2000'], grp['DEJ2000'], grp['angRad']))
    
    # make query
    tstart = time.process_time()
    tbl = SDSS.query_sql(query, data_release=15)
    
    # save table to csv file 
    ofile = "phot_sdss_grps/phot_gals_irgp_{}.csv".format(grp['iGrID'])
    tbl.write(ofile, format="ascii.csv", overwrite=True)
    print("    Extracted {} galaxies in {:.1f} sec, saved to {}".format(len(tbl), time.process_time() - tstart, ofile))

[1/479] Working on iGrID = 51       (Ntot = 67  Rad = 258.8 arcmin) at RA=57.41119385; DEC=0.204412
    Extracted 3534 galaxies in 0.4 sec, saved to phot_sdss_grps/phot_gals_irgp_51.csv
[2/479] Working on iGrID = 142      (Ntot = 51  Rad = 194.7 arcmin) at RA=-1.79300356; DEC=0.192726
    Extracted 3094 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_142.csv
[3/479] Working on iGrID = 157      (Ntot = 34  Rad = 195.3 arcmin) at RA=-3.4479351; DEC=0.128148
    Extracted 1054 galaxies in 0.0 sec, saved to phot_sdss_grps/phot_gals_irgp_157.csv
[4/479] Working on iGrID = 184      (Ntot = 44  Rad = 10.8 arcmin) at RA=15.2066021; DEC=0.130605
    Extracted 1100 galaxies in 0.0 sec, saved to phot_sdss_grps/phot_gals_irgp_184.csv
[5/479] Working on iGrID = 211      (Ntot = 91  Rad = 195.7 arcmin) at RA=-2.52054167; DEC=0.184561
    Extracted 2478 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_211.csv
[6/479] Working on iGrID = 362      (Ntot = 35  Rad = 27.3 arcmin) at

    Extracted 8831 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_11499.csv
[48/479] Working on iGrID = 11737    (Ntot = 34  Rad = 155.6 arcmin) at RA=2.36079955; DEC=0.143046
    Extracted 1677 galaxies in 0.0 sec, saved to phot_sdss_grps/phot_gals_irgp_11737.csv
[49/479] Working on iGrID = 12324    (Ntot = 125 Rad = 168.5 arcmin) at RA=2.48143363; DEC=0.312153
    Extracted 7406 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_12324.csv
[50/479] Working on iGrID = 12639    (Ntot = 48  Rad = 206.2 arcmin) at RA=2.86194038; DEC=0.147773
    Extracted 2144 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_12639.csv
[51/479] Working on iGrID = 12987    (Ntot = 70  Rad = 170.4 arcmin) at RA=2.88725781; DEC=0.166068
    Extracted 2379 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_12987.csv
[52/479] Working on iGrID = 13370    (Ntot = 220 Rad = 220.2 arcmin) at RA=3.46542287; DEC=0.61668
    Extracted 28502 galaxies in 0.6 sec, saved to phot_s

    Extracted 4498 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_30247.csv
[92/479] Working on iGrID = 30251    (Ntot = 82  Rad = 179.4 arcmin) at RA=5.08905125; DEC=0.183982
    Extracted 3355 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_30251.csv
[93/479] Working on iGrID = 30259    (Ntot = 31  Rad = 177.5 arcmin) at RA=5.70080709; DEC=0.097611
    Extracted 734 galaxies in 0.0 sec, saved to phot_sdss_grps/phot_gals_irgp_30259.csv
[94/479] Working on iGrID = 30473    (Ntot = 64  Rad = 182.6 arcmin) at RA=5.38603687; DEC=0.18472
    Extracted 2869 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_30473.csv
[95/479] Working on iGrID = 30772    (Ntot = 65  Rad = 175.3 arcmin) at RA=5.73473024; DEC=0.172637
    Extracted 2768 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_30772.csv
[96/479] Working on iGrID = 30875    (Ntot = 40  Rad = 176.0 arcmin) at RA=5.7981863; DEC=0.088254
    Extracted 696 galaxies in 0.0 sec, saved to phot_sdss_

    Extracted 4663 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_46404.csv
[136/479] Working on iGrID = 47499    (Ntot = 41  Rad = 23.7 arcmin) at RA=-0.66498792; DEC=0.150924
    Extracted 1947 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_47499.csv
[137/479] Working on iGrID = 47811    (Ntot = 55  Rad = 229.7 arcmin) at RA=6.23225355; DEC=0.161471
    Extracted 2642 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_47811.csv
[138/479] Working on iGrID = 47822    (Ntot = 45  Rad = 261.8 arcmin) at RA=58.48546219; DEC=0.203291
    Extracted 2620 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_47822.csv
[139/479] Working on iGrID = 48277    (Ntot = 40  Rad = 22.9 arcmin) at RA=0.55596983; DEC=0.112922
    Extracted 1536 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_48277.csv
[140/479] Working on iGrID = 49053    (Ntot = 51  Rad = 41.7 arcmin) at RA=-0.57077891; DEC=0.165164
    Extracted 2796 galaxies in 0.2 sec, saved to p

    Extracted 13189 galaxies in 0.4 sec, saved to phot_sdss_grps/phot_gals_irgp_61254.csv
[180/479] Working on iGrID = 62024    (Ntot = 93  Rad = 127.1 arcmin) at RA=30.43132019; DEC=0.146149
    Extracted 1551 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_62024.csv
[181/479] Working on iGrID = 62551    (Ntot = 34  Rad = 118.0 arcmin) at RA=23.48302078; DEC=0.186469
    Extracted 2159 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_62551.csv
[182/479] Working on iGrID = 64526    (Ntot = 32  Rad = 176.9 arcmin) at RA=7.01451159; DEC=0.123498
    Extracted 1369 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_64526.csv
[183/479] Working on iGrID = 64669    (Ntot = 87  Rad = 175.7 arcmin) at RA=8.16227913; DEC=0.35827
    Extracted 11467 galaxies in 0.4 sec, saved to phot_sdss_grps/phot_gals_irgp_64669.csv
[184/479] Working on iGrID = 65627    (Ntot = 110 Rad = 186.9 arcmin) at RA=8.82488155; DEC=0.180053
    Extracted 2847 galaxies in 0.2 sec, saved t

...

    Extracted 2523 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_99693.csv
[266/479] Working on iGrID = 99755    (Ntot = 45  Rad = 235.7 arcmin) at RA=8.24110985; DEC=0.297167
    Extracted 8098 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_99755.csv
[267/479] Working on iGrID = 100041   (Ntot = 47  Rad = 214.2 arcmin) at RA=10.8074255; DEC=0.21045
    Extracted 3576 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_100041.csv
[268/479] Working on iGrID = 101211   (Ntot = 156 Rad = 215.5 arcmin) at RA=48.57073975; DEC=0.27767
    Extracted 7855 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_101211.csv
[269/479] Working on iGrID = 101670   (Ntot = 49  Rad = 226.9 arcmin) at RA=8.49569988; DEC=0.25725
    Extracted 6077 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_101670.csv
[270/479] Working on iGrID = 101773   (Ntot = 181 Rad = 167.9 arcmin) at RA=40.820755; DEC=0.329344
    Extracted 8066 galaxies in 0.2 sec, saved to p

    Extracted 13430 galaxies in 0.5 sec, saved to phot_sdss_grps/phot_gals_irgp_111261.csv
[309/479] Working on iGrID = 112271   (Ntot = 250 Rad = 239.6 arcmin) at RA=27.23340988; DEC=0.240063
    Extracted 6237 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_112271.csv
[310/479] Working on iGrID = 115311   (Ntot = 31  Rad = 251.1 arcmin) at RA=26.68275452; DEC=0.212108
    Extracted 3452 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_115311.csv
[311/479] Working on iGrID = 115573   (Ntot = 61  Rad = 250.7 arcmin) at RA=27.44371223; DEC=0.166677
    Extracted 2121 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_115573.csv
[312/479] Working on iGrID = 116091   (Ntot = 77  Rad = 239.5 arcmin) at RA=28.79562759; DEC=0.252447
    Extracted 5759 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_116091.csv
[313/479] Working on iGrID = 116875   (Ntot = 70  Rad = 241.1 arcmin) at RA=33.32640839; DEC=0.333935
    Extracted 9099 galaxies in 0.3 sec,

    Extracted 1365 galaxies in 0.0 sec, saved to phot_sdss_grps/phot_gals_irgp_136677.csv
[352/479] Working on iGrID = 137707   (Ntot = 32  Rad = 172.4 arcmin) at RA=36.63851166; DEC=0.150775
    Extracted 2656 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_137707.csv
[353/479] Working on iGrID = 138711   (Ntot = 38  Rad = 204.2 arcmin) at RA=34.84152985; DEC=0.230938
    Extracted 4697 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_138711.csv
[354/479] Working on iGrID = 138916   (Ntot = 37  Rad = 203.9 arcmin) at RA=36.00130081; DEC=0.205019
    Extracted 3890 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_138916.csv
[355/479] Working on iGrID = 139785   (Ntot = 89  Rad = 203.6 arcmin) at RA=34.77776718; DEC=0.692662
    Extracted 42591 galaxies in 0.9 sec, saved to phot_sdss_grps/phot_gals_irgp_139785.csv
[356/479] Working on iGrID = 142208   (Ntot = 33  Rad = 197.8 arcmin) at RA=34.41804504; DEC=0.285118
    Extracted 7034 galaxies in 0.2 sec,

    Extracted 6898 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_152483.csv
[395/479] Working on iGrID = 152780   (Ntot = 59  Rad = 207.3 arcmin) at RA=28.10731697; DEC=0.163558
    Extracted 2090 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_152780.csv
[396/479] Working on iGrID = 152805   (Ntot = 62  Rad = 209.8 arcmin) at RA=28.01791763; DEC=0.166867
    Extracted 3041 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_152805.csv
[397/479] Working on iGrID = 153044   (Ntot = 31  Rad = 229.9 arcmin) at RA=19.64247131; DEC=0.149397
    Extracted 2159 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_153044.csv
[398/479] Working on iGrID = 153084   (Ntot = 765 Rad = 195.0 arcmin) at RA=27.97694016; DEC=0.717534
    Extracted 44686 galaxies in 1.0 sec, saved to phot_sdss_grps/phot_gals_irgp_153084.csv
[399/479] Working on iGrID = 153143   (Ntot = 38  Rad = 207.1 arcmin) at RA=25.67529106; DEC=0.113694
    Extracted 1217 galaxies in 0.0 sec,

    Extracted 11628 galaxies in 0.3 sec, saved to phot_sdss_grps/phot_gals_irgp_174811.csv
[438/479] Working on iGrID = 176059   (Ntot = 33  Rad = 212.8 arcmin) at RA=19.18362427; DEC=0.15906
    Extracted 2283 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_176059.csv
[439/479] Working on iGrID = 176164   (Ntot = 179 Rad = 223.2 arcmin) at RA=16.70285797; DEC=0.334553
    Extracted 10704 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_176164.csv
[440/479] Working on iGrID = 176410   (Ntot = 41  Rad = 137.1 arcmin) at RA=16.04655075; DEC=0.145905
    Extracted 2136 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_176410.csv
[441/479] Working on iGrID = 176411   (Ntot = 32  Rad = 139.5 arcmin) at RA=16.19809151; DEC=0.397531
    Extracted 12990 galaxies in 0.3 sec, saved to phot_sdss_grps/phot_gals_irgp_176411.csv
[442/479] Working on iGrID = 176427   (Ntot = 34  Rad = 208.8 arcmin) at RA=20.41714859; DEC=0.201075
    Extracted 3769 galaxies in 0.1 sec